# All Data To One Table

***

This will place all the rows from all the element-summary>history files into one table.

#### Import Packages

In [1]:
import pandas as pd
import numpy as np
import time
import collections
import json

#### Read all element-summary files

Initialise loop parameters and run loop.

In [2]:
# number of players
# hardcoded as this is fixed and saves time counting files.
nPlayers = 624

# initialise count.
# this will increase for each player's fixture
count = 0;

# get column names
# bool which changes to false after one step of the loop to get column headers once.
gcn = True

# Empty ordered dictionary file
# all player's fixtures will be stored here
data_dict = collections.OrderedDict()

# Start timer
start_time = time.time()

# Loop through all players
for i in range(1,nPlayers+1):
    
    # Open json file
    with open('../../data/json/element-summary/' + str(i) + '.json') as f:
        es = json.load(f)
        
    # Number of fixtures for this player
    nFixtures = len(es["history"])
    
    # Get column names. only do this once
    if gcn:
        cn = es["history"][0].keys()
        gcn = False
    
    # Loop through this players fixtures
    for m in range(0, nFixtures):
        a = es["history"][m]
        data_dict[count] = a
        count = count+1

data = pd.DataFrame.from_dict(data_dict, orient="index")
print("Time Taken: %s seconds" % (time.time() - start_time))

# View DataFrame
data.head()

Time Taken: 1.5827109813690186 seconds


,id,kickoff_time,kickoff_time_formatted,team_h_score,team_a_score,was_home,round,total_points,value,transfers_balance,...,errors_leading_to_goal,errors_leading_to_goal_attempt,tackled,offside,target_missed,fouls,dribbles,element,fixture,opponent_team
0,1,2018-08-12T15:00:00Z,12 Aug 16:00,0,2,True,1,3,50,0,...,0,0,0,0,0,0,0,1,1,13
1,526,2018-08-18T16:30:00Z,18 Aug 17:30,3,2,False,2,3,50,188,...,0,0,0,0,0,0,0,1,14,6
2,1053,2018-08-25T14:00:00Z,25 Aug 15:00,3,1,True,3,3,50,8540,...,0,0,0,0,0,0,0,1,21,19
3,1584,2018-09-02T12:30:00Z,02 Sep 13:30,2,3,False,4,1,50,9582,...,0,2,0,0,0,0,0,1,33,5
4,2121,2018-09-15T14:00:00Z,15 Sep 15:00,1,2,False,5,2,50,-3297,...,0,0,0,0,0,0,0,1,46,15


Drop columns with no use. First, take a look at all the columns.

In [3]:
print(data.columns)

Index(['id', 'kickoff_time', 'kickoff_time_formatted', 'team_h_score',
       'team_a_score', 'was_home', 'round', 'total_points', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out',
       'loaned_in', 'loaned_out', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 'threat', 'ict_index', 'ea_index',
       'open_play_crosses', 'big_chances_created',
       'clearances_blocks_interceptions', 'recoveries', 'key_passes',
       'tackles', 'winning_goals', 'attempted_passes', 'completed_passes',
       'penalties_conceded', 'big_chances_missed', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'tackled', 'offside', 'target_missed',
       'fouls', 'dribbles', 'element', 'fixture', 'opponent_team'],
      dtype='object')


In [4]:
# There are fewer columns to keep than to drop, so define ones to keep.
columns_to_keep = ['element', 'round', 'kickoff_time', 'fixture', 'opponent_team', 'was_home', 'minutes', 'total_points']
data_clean = data[columns_to_keep].copy()

# Restart indexing from 0
data_clean['element'] = data_clean['element']-1
data_clean['fixture'] = data_clean['fixture']-1
data_clean['opponent_team'] = data_clean['opponent_team']-1

# Look at new table
data_clean.head()

,element,round,kickoff_time,fixture,opponent_team,was_home,minutes,total_points
0,0,1,2018-08-12T15:00:00Z,0,12,True,90,3
1,0,2,2018-08-18T16:30:00Z,13,5,False,90,3
2,0,3,2018-08-25T14:00:00Z,20,18,True,90,3
3,0,4,2018-09-02T12:30:00Z,32,4,False,90,1
4,0,5,2018-09-15T14:00:00Z,45,14,False,90,2


In [5]:
# sort first by element number, and then by kickoff time
data_clean = data_clean.sort_values(by=['element', 'kickoff_time']).reset_index(drop=True)
data_clean.head()

,element,round,kickoff_time,fixture,opponent_team,was_home,minutes,total_points
0,0,1,2018-08-12T15:00:00Z,0,12,True,90,3
1,0,2,2018-08-18T16:30:00Z,13,5,False,90,3
2,0,3,2018-08-25T14:00:00Z,20,18,True,90,3
3,0,4,2018-09-02T12:30:00Z,32,4,False,90,1
4,0,5,2018-09-15T14:00:00Z,45,14,False,90,2


#### Read fixture information

Read fixture json and drop blaoting columns.

In [6]:
with open('../../data/json/fixtures/fixtures.json', 'r') as f:
    fixtures = pd.DataFrame(json.load(f))

fixtures = fixtures.drop(columns=['stats'])
fixtures = fixtures.sort_values(by=['id']).reset_index(drop=True)
fixtures['id'] = fixtures['id']-1
fixtures['team_h'] = fixtures['team_h']-1
fixtures['team_a'] = fixtures['team_a']-1

fixtures.head()

,code,deadline_time,deadline_time_formatted,event,event_day,finished,finished_provisional,id,kickoff_time,kickoff_time_formatted,minutes,provisional_start_time,started,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score
0,987593,2018-08-10T18:00:00Z,10 Aug 19:00,1,3,True,True,0,2018-08-12T15:00:00Z,12 Aug 16:00,90,False,True,12,4,2,0,4,0
1,987592,2018-08-10T18:00:00Z,10 Aug 19:00,1,2,True,True,1,2018-08-11T14:00:00Z,11 Aug 15:00,90,False,True,4,3,0,1,2,2
2,987594,2018-08-10T18:00:00Z,10 Aug 19:00,1,2,True,True,2,2018-08-11T14:00:00Z,11 Aug 15:00,90,False,True,6,2,2,8,2,0
3,987595,2018-08-10T18:00:00Z,10 Aug 19:00,1,2,True,True,3,2018-08-11T14:00:00Z,11 Aug 15:00,90,False,True,5,2,3,9,4,0
4,987596,2018-08-10T18:00:00Z,10 Aug 19:00,1,3,True,True,4,2018-08-12T12:30:00Z,12 Aug 13:30,90,False,True,18,5,0,11,3,4


#### Read element information to get player positions

In [7]:
with open('../../data/json/elements/elements.json', 'r') as f:
    elements = pd.DataFrame(json.load(f))

print(elements.columns)
elements = elements.sort_values(by=['id']).reset_index(drop=True)
elements['id'] = elements['id']-1
elements['element_type'] = elements['element_type']-1
elements.head()

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count', 'ea_index',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'loaned_in', 'loaned_out', 'loans_in',
       'loans_out', 'minutes', 'news', 'news_added', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'red_cards', 'saves', 'second_name', 'selected_by_percent', 'special',
       'squad_number', 'status', 'team', 'team_code', 'threat', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'yellow_cards'],
      dtype='object')


,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,3,130,100.0,100.0,1,11334,0,0,-3,...,0.0,24,83497,0,136211,0,0.0,5.1,Cech,0
1,0,5,568,NaN,NaN,6,80201,0,0,-1,...,0.0,106,339095,0,250834,0,0.5,21.6,Leno,0
2,0,8,319,100.0,100.0,3,51507,0,0,-1,...,105.0,62,128478,0,92187,0,0.1,11.5,Koscielny,1
3,5,5,304,0.0,0.0,4,98745,0,0,-2,...,280.0,60,567084,0,1143684,0,0.0,11.3,Bellerín,3
4,4,7,392,100.0,100.0,5,38411,0,0,-1,...,224.0,77,298216,0,290921,0,0.3,14.3,Monreal,5


Loop through __data_clean__ rows and add the elements team to DataFrame.

In [8]:
# number of rows
nrows = len(data_clean)

# team
team = np.zeros(nrows, dtype='int')
element_type = np.zeros(nrows, dtype='int')
name = ['']*nrows

for i in range(nrows):
    
    # Get current element and fixture
    current_fixture = data_clean['fixture'][i]
    current_element = data_clean['element'][i]
    
    # Add element type to array
    element_type[i] = elements['element_type'][current_element]
    
    # add team to array
    if data_clean['was_home'][i]:
        team[i] = fixtures['team_h'][current_fixture]
    else:
        team[i] = fixtures['team_a'][current_fixture]
        
    # add player name to list
    name[i] = elements['web_name'][current_element]
        
        
# Add new columns to dataframe
data_clean['team'] = pd.Series(team)
data_clean['element_type'] = pd.Series(element_type)
data_clean['name'] = pd.Series(name)
data_clean.head()

,element,round,kickoff_time,fixture,opponent_team,was_home,minutes,total_points,team,element_type,name
0,0,1,2018-08-12T15:00:00Z,0,12,True,90,3,0,0,Cech
1,0,2,2018-08-18T16:30:00Z,13,5,False,90,3,0,0,Cech
2,0,3,2018-08-25T14:00:00Z,20,18,True,90,3,0,0,Cech
3,0,4,2018-09-02T12:30:00Z,32,4,False,90,1,0,0,Cech
4,0,5,2018-09-15T14:00:00Z,45,14,False,90,2,0,0,Cech


In [9]:
data_clean = data_clean[['name', 'element', 'element_type', 'round', 'kickoff_time', 'fixture', 'team', 'opponent_team', 'was_home', 'minutes', 'total_points']]
data_clean[1000:2000].head(100)

,name,element,element_type,round,kickoff_time,fixture,team,opponent_team,was_home,minutes,total_points
1000,Steve Cook,26,1,13,2018-11-25T13:30:00Z,120,1,0,True,90,1
1001,Steve Cook,26,1,14,2018-12-01T15:00:00Z,137,1,12,False,90,1
1002,Steve Cook,26,1,15,2018-12-04T19:45:00Z,140,1,9,True,90,2
1003,Steve Cook,26,1,16,2018-12-08T12:30:00Z,151,1,11,True,90,-2
1004,Steve Cook,26,1,17,2018-12-15T15:00:00Z,169,1,19,False,90,0
1005,Steve Cook,26,1,18,2018-12-22T15:00:00Z,171,1,2,True,90,7
1006,Steve Cook,26,1,19,2018-12-26T15:00:00Z,188,1,16,False,90,0
1007,Steve Cook,26,1,20,2018-12-30T16:30:00Z,196,1,13,False,90,0
1008,Steve Cook,26,1,21,2019-01-02T19:45:00Z,201,1,17,True,90,1
1009,Steve Cook,26,1,22,2019-01-13T14:15:00Z,215,1,7,False,90,1


In [10]:
data_clean.to_csv(r'../../data/csv/data01.csv', index=False, index_label=False)